In [4]:
!pip install kaggle mlxtend networkx matplotlib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "./"  

In [ ]:
0!kaggle datasets download -d grouplens/movielens-latest-small

Dataset URL: https://www.kaggle.com/datasets/grouplens/movielens-latest-small
License(s): other




  0%|          | 0.00/971k [00:00<?, ?B/s]
100%|██████████| 971k/971k [00:00<00:00, 1.25MB/s]
100%|██████████| 971k/971k [00:00<00:00, 1.25MB/s]


In [7]:
import zipfile

with zipfile.ZipFile("movielens-latest-small.zip","r") as zip_ref:
    zip_ref.extractall("movielens_small")

In [8]:
import pandas as pd

movies = pd.read_csv("movielens_small/movies.csv")
ratings = pd.read_csv("movielens_small/ratings.csv")

print("Movies Sample:")
print(movies.head())

print("\nRatings Sample:")
print(ratings.head())

Movies Sample:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings Sample:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [ ]:
watched = ratings[ratings['rating'] >= 3.5]

movies['genres'] = movies['genres'].str.split('|')

data = watched.merge(movies, on='movieId')

transactions = data.groupby('userId')['genres'].apply(lambda x: list(x.explode())).tolist()

print(f"Number of transactions (users): {len(transactions)}")

Number of transactions (users): 609


In [10]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_array, columns=te.columns_)

df.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,True,True,True,True,True,False,True,True,True,True,False,True,True,True,True,True,True,True
1,False,True,True,False,False,True,True,True,True,False,False,False,True,False,True,True,True,True,True,True
2,False,True,True,False,False,True,False,False,True,True,False,True,False,False,True,False,True,True,False,False
3,False,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True
4,False,True,True,True,True,True,True,False,True,True,False,False,True,True,True,True,False,True,True,True


In [11]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
print("Frequent Itemsets (support >= 0.05):")
print(frequent_itemsets.head())

Frequent Itemsets (support >= 0.05):
    support     itemsets
0  0.978654     (Action)
1  0.965517  (Adventure)
2  0.752053  (Animation)
3  0.791461   (Children)
4  0.990148     (Comedy)


In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)
rules = rules.sort_values(by='lift', ascending=False)

print("Top 10 Association Rules:")
print(rules[['antecedents','consequents','support','confidence','lift']].head(10))

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

top_rules = rules.head(10)
G = nx.DiGraph()

# Add edges with lift as weight
for _, row in top_rules.iterrows():
    for antecedent in row['antecedents']:
        for consequent in row['consequents']:
            G.add_edge(antecedent, consequent, weight=row['lift'])

plt.figure(figsize=(10,6))
pos = nx.spring_layout(G, k=1)
edges = G.edges()
weights = [G[u][v]['weight'] for u,v in edges]
nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=3000,
        edge_color=weights, width=3, edge_cmap=plt.cm.viridis)
plt.title("Top Movie Genre Association Rules")
plt.show()